# Advanced indexing

In [1]:
import sys
sys.path.insert(0, '..')
import zarr
import numpy as np
np.random.seed(42)
import cProfile
zarr.__version__

'2.1.5.dev113'

## Functionality and API

### Indexing a 1D array with a Boolean (mask) array

Supported via ``get/set_mask_selection()`` and ``.vindex[]``. Also supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``.

In [2]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [False,  True,  False,  True, False, True, False,  True,  False,  True]

In [3]:
# get items
za.vindex[ix]

array([1, 3, 5, 7, 9])

In [4]:
# get items
za.oindex[ix]

array([1, 3, 5, 7, 9])

In [5]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

In [6]:
# set items
za.oindex[ix] = a[ix] * 100
za[:]

array([  0, 100,   2, 300,   4, 500,   6, 700,   8, 900])

In [7]:
# if using .oindex, indexing array can be any array-like, e.g., Zarr array
zix = zarr.array(ix, chunks=2)
za = zarr.array(a, chunks=2)
za.oindex[zix]  # will not load all zix into memory

array([1, 3, 5, 7, 9])

### Indexing a 1D array with a 1D integer (coordinate) array

Supported via ``get/set_coordinate_selection()`` and ``.vindex[]``. Also supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``.

In [8]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [1, 3, 5, 7, 9]

In [9]:
# get items
za.vindex[ix]

array([1, 3, 5, 7, 9])

In [10]:
# get items
za.oindex[ix]

array([1, 3, 5, 7, 9])

In [11]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

In [12]:
# set items
za.oindex[ix] = a[ix] * 100
za[:]

array([  0, 100,   2, 300,   4, 500,   6, 700,   8, 900])

### Indexing a 1D array with a multi-dimensional integer (coordinate) array

Supported via ``get/set_coordinate_selection()`` and ``.vindex[]``.

In [13]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = np.array([[1, 3, 5], [2, 4, 6]])

In [14]:
# get items
za.vindex[ix]

array([[1, 3, 5],
       [2, 4, 6]])

In [15]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10, 20, 30, 40, 50, 60,  7,  8,  9])

### Slicing a 1D array with step > 1

Slices with step > 1 are supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``. Internally these are converted to an integer array via ``np.arange``.

In [16]:
a = np.arange(10)
za = zarr.array(a, chunks=2)

In [17]:
# get items
za.oindex[1::2]

array([1, 3, 5, 7, 9])

In [18]:
# get items with negative step
za.oindex[-1::-2]

array([9, 7, 5, 3, 1])

In [19]:
# set items
za.oindex[1::2] = a[1::2] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

### Orthogonal (outer) indexing of multi-dimensional arrays

Orthogonal (a.k.a. outer) indexing is supported with either Boolean or integer arrays, in combination with integers and slices. This functionality is provided via the ``get/set_orthogonal_selection()`` methods. For convenience, this functionality is also available via the ``.oindex[]`` property.

In [20]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [21]:
# orthogonal indexing with Boolean arrays
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [22]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [23]:
# orthogonal indexing with integer arrays
ix0 = [1, 3]
ix1 = [0, 2]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [24]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [25]:
# combine with slice
za.oindex[[1,  3], :]

array([[ 3,  4,  5],
       [ 9, 10, 11]])

In [26]:
# combine with slice
za.oindex[:, [0, 2]]

array([[ 0,  2],
       [ 3,  5],
       [ 6,  8],
       [ 9, 11],
       [12, 14]])

In [27]:
# set items via Boolean selection
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
selection = ix0, ix1
value = 42
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [42,  4, 42],
       [ 6,  7,  8],
       [42, 10, 42],
       [12, 13, 14]])

In [28]:
# alternative API
za.oindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4, 44],
       [ 6,  7,  8],
       [44, 10, 44],
       [12, 13, 14]])

In [29]:
# set items via integer selection
ix0 = [1, 3]
ix1 = [0, 2]
selection = ix0, ix1
value = 46
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [46,  4, 46],
       [ 6,  7,  8],
       [46, 10, 46],
       [12, 13, 14]])

In [30]:
# alternative API
za.oindex[ix0, ix1] = 48
za[:]

array([[ 0,  1,  2],
       [48,  4, 48],
       [ 6,  7,  8],
       [48, 10, 48],
       [12, 13, 14]])

### Coordinate indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by indexing with integer (coordinate) arrays is supported. This functionality is provided via the ``get/set_coordinate_selection()`` methods. For convenience, this functionality is also available via the ``.vindex[]`` property.

In [31]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [32]:
# get items
ix0 = [1, 3]
ix1 = [0, 2]
za.get_coordinate_selection((ix0, ix1))

array([ 3, 11])

In [33]:
# alternative API
za.vindex[ix0, ix1]

array([ 3, 11])

In [34]:
# set items
za.set_coordinate_selection((ix0, ix1), 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [35]:
# alternative API
za.vindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

### Mask indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by a Boolean array is supported. This functionality is provided via the ``get/set_mask_selection()`` methods. For convenience, this functionality is also available via the ``.vindex[]`` property.

In [36]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [37]:
ix = np.zeros_like(a, dtype=bool)
ix[1, 0] = True
ix[3, 2] = True
za.get_mask_selection(ix)

array([ 3, 11])

In [38]:
za.vindex[ix]

array([ 3, 11])

In [39]:
za.set_mask_selection(ix, 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [40]:
za.vindex[ix] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

## 1D Benchmarking

In [41]:
c = np.arange(100000000)
c.nbytes

800000000

In [42]:
%time zc = zarr.array(c)
zc.info

CPU times: user 392 ms, sys: 84 ms, total: 476 ms
Wall time: 124 ms


Type,zarr.core.Array
Data type,int64
Shape,"(100000000,)"
Chunk shape,"(97657,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,11854081 (11.3M)
Storage ratio,67.5


In [43]:
%time c.copy()

CPU times: user 88 ms, sys: 52 ms, total: 140 ms
Wall time: 143 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [44]:
%time zc[:]

CPU times: user 408 ms, sys: 88 ms, total: 496 ms
Wall time: 213 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

### bool dense selection

In [45]:
# relatively dense selection - 10%
ix_dense_bool = np.random.binomial(1, 0.1, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_dense_bool)

9997476

In [46]:
%time c[ix_dense_bool]

CPU times: user 256 ms, sys: 4 ms, total: 260 ms
Wall time: 258 ms


array([       1,       11,       33, ..., 99999988, 99999989, 99999990])

In [47]:
%time zc.oindex[ix_dense_bool]

CPU times: user 864 ms, sys: 108 ms, total: 972 ms
Wall time: 439 ms


array([       1,       11,       33, ..., 99999988, 99999989, 99999990])

In [48]:
%time zc.vindex[ix_dense_bool]

CPU times: user 808 ms, sys: 32 ms, total: 840 ms
Wall time: 564 ms


array([       1,       11,       33, ..., 99999988, 99999989, 99999990])

In [49]:
cProfile.run('zc.oindex[ix_dense_bool]', sort='time')

         67653 function calls in 0.490 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1025    0.201    0.000    0.201    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.158    0.000    0.167    0.000 core.py:997(_decode_chunk)
     1024    0.047    0.000    0.234    0.000 core.py:822(_chunk_getitem)
     1024    0.012    0.000    0.012    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1025    0.007    0.000    0.238    0.000 indexing.py:484(__iter__)
     1024    0.006    0.000    0.212    0.000 index_tricks.py:26(ix_)
     2048    0.005    0.000    0.005    0.000 core.py:332(<genexpr>)
     2048    0.005    0.000    0.005    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     7180    0.004    0.000    0.009    0.000 {built-in method builtins.isinstance}
        1    0.004    0.004    0.476    0.476 core.py:576(_get_selection)
     1024    0.004    0.000    0.004    0.000 {bui

Method ``nonzero`` is being called internally within numpy to convert bool to int selections, no way to avoid.

In [50]:
cProfile.run('zc.vindex[ix_dense_bool]', sort='time')

         43161 function calls in 0.575 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.215    0.108    0.215    0.108 {method 'nonzero' of 'numpy.ndarray' objects}
        2    0.094    0.047    0.094    0.047 indexing.py:580(<genexpr>)
     1024    0.093    0.000    0.098    0.000 core.py:997(_decode_chunk)
     1024    0.042    0.000    0.157    0.000 core.py:822(_chunk_getitem)
        1    0.028    0.028    0.028    0.028 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
        1    0.021    0.021    0.179    0.179 indexing.py:553(__init__)
     2048    0.011    0.000    0.011    0.000 indexing.py:645(<genexpr>)
        1    0.010    0.010    0.010    0.010 function_base.py:1848(diff)
     1025    0.008    0.000    0.021    0.000 indexing.py:624(__iter__)
     2054    0.003    0.000    0.003    0.000 co

``.vindex[]`` is a bit slower, possibly because internally it converts to a coordinate array first.

### int dense selection

In [51]:
ix_dense_int = np.random.choice(c.shape[0], size=c.shape[0]//10, replace=True)
ix_dense_int_sorted = ix_dense_int.copy()
ix_dense_int_sorted.sort()
len(ix_dense_int)

10000000

In [52]:
%time c[ix_dense_int_sorted]

CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 64.4 ms


array([       0,       33,       42, ..., 99999987, 99999994, 99999999])

In [53]:
%time zc.oindex[ix_dense_int_sorted]

CPU times: user 560 ms, sys: 52 ms, total: 612 ms
Wall time: 354 ms


array([       0,       33,       42, ..., 99999987, 99999994, 99999999])

In [54]:
%time zc.vindex[ix_dense_int_sorted]

CPU times: user 588 ms, sys: 60 ms, total: 648 ms
Wall time: 367 ms


array([       0,       33,       42, ..., 99999987, 99999994, 99999999])

In [55]:
%time c[ix_dense_int]

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 123 ms


array([23268249, 15578653,     7864, ..., 68558269,  7682216, 66838288])

In [56]:
%time zc.oindex[ix_dense_int]

CPU times: user 1.94 s, sys: 68 ms, total: 2.01 s
Wall time: 1.71 s


array([23268249, 15578653,     7864, ..., 68558269,  7682216, 66838288])

In [57]:
%time zc.vindex[ix_dense_int]

CPU times: user 1.99 s, sys: 76 ms, total: 2.06 s
Wall time: 1.71 s


array([23268249, 15578653,     7864, ..., 68558269,  7682216, 66838288])

In [58]:
cProfile.run('zc.oindex[ix_dense_int_sorted]', sort='time')

         64607 function calls in 0.380 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.135    0.135    0.172    0.172 indexing.py:304(__init__)
     1024    0.090    0.000    0.095    0.000 core.py:997(_decode_chunk)
     1024    0.043    0.000    0.152    0.000 core.py:822(_chunk_getitem)
     1025    0.026    0.000    0.027    0.000 indexing.py:351(__iter__)
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
        1    0.011    0.011    0.011    0.011 function_base.py:1848(diff)
     1025    0.006    0.000    0.052    0.000 indexing.py:484(__iter__)
     1024    0.004    0.000    0.007    0.000 index_tricks.py:26(ix_)
     2048    0.004    0.000    0.004    0.000 core.py:332(<genexpr>)
        1    0.003    0.003    0.381    0.381 core.py:531(get_orthogonal_selection)
     2048    0.003    0.000    0.003    0.000 {method 'reshape' of 'numpy.ndarray' objects}
   

In [59]:
cProfile.run('zc.vindex[ix_dense_int_sorted]', sort='time')

         43143 function calls in 0.372 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.104    0.052    0.104    0.052 indexing.py:580(<genexpr>)
     1024    0.090    0.000    0.095    0.000 core.py:997(_decode_chunk)
     1024    0.042    0.000    0.154    0.000 core.py:822(_chunk_getitem)
        1    0.028    0.028    0.028    0.028 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.026    0.026    0.194    0.194 indexing.py:553(__init__)
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
     2048    0.011    0.000    0.011    0.000 indexing.py:645(<genexpr>)
        1    0.010    0.010    0.010    0.010 function_base.py:1848(diff)
     1025    0.008    0.000    0.021    0.000 indexing.py:624(__iter__)
     2054    0.003    0.000    0.003    0.000 core.py:332(<genexpr>)
        4    0.003    0.001    0.003    0.001 {method 'reduce' of 'nump

In [60]:
cProfile.run('zc.oindex[ix_dense_int]', sort='time')

         85095 function calls in 1.770 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.123    1.123    1.123    1.123 {method 'argsort' of 'numpy.ndarray' objects}
        1    0.134    0.134    1.417    1.417 indexing.py:304(__init__)
     1024    0.134    0.000    0.273    0.000 core.py:822(_chunk_getitem)
        1    0.122    0.122    0.122    0.122 {method 'take' of 'numpy.ndarray' objects}
     1024    0.117    0.000    0.123    0.000 core.py:997(_decode_chunk)
     1025    0.032    0.000    0.033    0.000 indexing.py:351(__iter__)
        1    0.025    0.025    0.025    0.025 {built-in method numpy.core.multiarray.bincount}
        1    0.010    0.010    0.010    0.010 function_base.py:1848(diff)
     1025    0.007    0.000    0.073    0.000 indexing.py:484(__iter__)
     2048    0.007    0.000    0.014    0.000 index_tricks.py:26(ix_)
     2048    0.005    0.000    0.005    0.000 core.py:332(<genexpr>

In [61]:
cProfile.run('zc.vindex[ix_dense_int]', sort='time')

         35981 function calls in 1.735 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.117    1.117    1.117    1.117 {method 'argsort' of 'numpy.ndarray' objects}
     1024    0.132    0.000    0.269    0.000 core.py:822(_chunk_getitem)
        2    0.121    0.061    0.121    0.061 indexing.py:604(<genexpr>)
     1024    0.116    0.000    0.122    0.000 core.py:997(_decode_chunk)
        2    0.108    0.054    0.108    0.054 indexing.py:580(<genexpr>)
        1    0.028    0.028    0.028    0.028 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.026    0.026    1.437    1.437 indexing.py:553(__init__)
        1    0.024    0.024    0.024    0.024 {built-in method numpy.core.multiarray.bincount}
     2048    0.012    0.000    0.012    0.000 indexing.py:645(<genexpr>)
        1    0.010    0.010    0.010    0.010 function_base.py:1848(diff)
     1025    0.009    0.000    0.024    0.000 i

When indices are not sorted, zarr needs to partially sort them so the occur in chunk order, so we only have to visit each chunk once. This sorting dominates the processing time and is unavoidable AFAIK.

### bool sparse selection

In [62]:
# relatively sparse selection
ix_sparse_bool = np.random.binomial(1, 0.0001, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_sparse_bool)

9932

In [63]:
%time c[ix_sparse_bool]

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 20 ms


array([     888,     9941,    15901, ..., 99988491, 99988714, 99995248])

In [64]:
%time zc.oindex[ix_sparse_bool]

CPU times: user 408 ms, sys: 28 ms, total: 436 ms
Wall time: 182 ms


array([     888,     9941,    15901, ..., 99988491, 99988714, 99995248])

In [65]:
%time zc.vindex[ix_sparse_bool]

CPU times: user 372 ms, sys: 36 ms, total: 408 ms
Wall time: 149 ms


array([     888,     9941,    15901, ..., 99988491, 99988714, 99995248])

In [66]:
cProfile.run('zc.oindex[ix_sparse_bool]', sort='time')

         67653 function calls in 0.186 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.097    0.000    0.103    0.000 core.py:997(_decode_chunk)
     1025    0.018    0.000    0.018    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.009    0.000    0.009    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1024    0.006    0.000    0.124    0.000 core.py:822(_chunk_getitem)
     1025    0.006    0.000    0.049    0.000 indexing.py:484(__iter__)
     1024    0.005    0.000    0.027    0.000 index_tricks.py:26(ix_)
     2048    0.004    0.000    0.004    0.000 core.py:332(<genexpr>)
     2048    0.003    0.000    0.003    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     7180    0.003    0.000    0.007    0.000 {built-in method builtins.isinstance}
     2049    0.003    0.000    0.004    0.000 abc.py:178(__instancecheck__)
        1    0.003    0.003    0.176    0.176 co

In [67]:
cProfile.run('zc.vindex[ix_sparse_bool]', sort='time')

         43161 function calls in 0.159 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.093    0.000    0.098    0.000 core.py:997(_decode_chunk)
        2    0.020    0.010    0.020    0.010 {method 'nonzero' of 'numpy.ndarray' objects}
     1025    0.008    0.000    0.015    0.000 indexing.py:624(__iter__)
     1024    0.006    0.000    0.122    0.000 core.py:822(_chunk_getitem)
     2048    0.005    0.000    0.005    0.000 indexing.py:645(<genexpr>)
     2054    0.003    0.000    0.003    0.000 core.py:332(<genexpr>)
        1    0.002    0.002    0.139    0.139 core.py:576(_get_selection)
     1024    0.002    0.000    0.005    0.000 arrayprint.py:381(wrapper)
     1026    0.002    0.000    0.002    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     1024    0.002    0.000    0.003    0.000 util.py:113(is_total_slice)
     1024    0.002    0.000    0.008    0.000 {method 'join' of 'str' objects}
     

### int sparse selection

In [68]:
ix_sparse_int = np.random.choice(c.shape[0], size=c.shape[0]//10000, replace=True)
ix_sparse_int_sorted = ix_sparse_int.copy()
ix_sparse_int_sorted.sort()
len(ix_sparse_int)

10000

In [69]:
%time c[ix_sparse_int_sorted]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 183 µs


array([   26607,    37803,    43822, ..., 99980305, 99994438, 99995776])

In [70]:
%time c[ix_sparse_int]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 189 µs


array([89501247, 55878596, 46320615, ..., 57048243,  1027560, 66644274])

In [71]:
%time zc.oindex[ix_sparse_int_sorted]

CPU times: user 384 ms, sys: 44 ms, total: 428 ms
Wall time: 166 ms


array([   26607,    37803,    43822, ..., 99980305, 99994438, 99995776])

In [72]:
%time zc.vindex[ix_sparse_int_sorted]

CPU times: user 376 ms, sys: 32 ms, total: 408 ms
Wall time: 137 ms


array([   26607,    37803,    43822, ..., 99980305, 99994438, 99995776])

In [73]:
%time zc.oindex[ix_sparse_int]

CPU times: user 412 ms, sys: 20 ms, total: 432 ms
Wall time: 174 ms


array([89501247, 55878596, 46320615, ..., 57048243,  1027560, 66644274])

In [74]:
%time zc.vindex[ix_sparse_int]

CPU times: user 360 ms, sys: 36 ms, total: 396 ms
Wall time: 134 ms


array([89501247, 55878596, 46320615, ..., 57048243,  1027560, 66644274])

In [75]:
cProfile.run('zc.oindex[ix_sparse_int]', sort='time')

         85095 function calls in 0.170 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.097    0.000    0.102    0.000 core.py:997(_decode_chunk)
     1025    0.006    0.000    0.044    0.000 indexing.py:484(__iter__)
     2048    0.006    0.000    0.013    0.000 index_tricks.py:26(ix_)
     1025    0.005    0.000    0.006    0.000 indexing.py:351(__iter__)
     1024    0.005    0.000    0.122    0.000 core.py:822(_chunk_getitem)
     2048    0.004    0.000    0.004    0.000 core.py:332(<genexpr>)
     3072    0.004    0.000    0.004    0.000 {method 'reshape' of 'numpy.ndarray' objects}
    10252    0.004    0.000    0.009    0.000 {built-in method builtins.isinstance}
     3073    0.004    0.000    0.006    0.000 abc.py:178(__instancecheck__)
     2048    0.003    0.000    0.025    0.000 indexing.py:377(ix_)
        1    0.003    0.003    0.169    0.169 core.py:576(_get_selection)
     1024    0.002    0.000

In [76]:
cProfile.run('zc.vindex[ix_sparse_int]', sort='time')

         35981 function calls in 0.136 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.093    0.000    0.098    0.000 core.py:997(_decode_chunk)
     1025    0.008    0.000    0.015    0.000 indexing.py:624(__iter__)
     1024    0.005    0.000    0.117    0.000 core.py:822(_chunk_getitem)
     2048    0.004    0.000    0.004    0.000 indexing.py:645(<genexpr>)
     2054    0.003    0.000    0.003    0.000 core.py:332(<genexpr>)
        1    0.002    0.002    0.135    0.135 core.py:576(_get_selection)
     1024    0.002    0.000    0.005    0.000 arrayprint.py:381(wrapper)
     1027    0.002    0.000    0.002    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     1024    0.002    0.000    0.008    0.000 {method 'join' of 'str' objects}
     1024    0.002    0.000    0.002    0.000 {built-in method numpy.core.multiarray.frombuffer}
     1024    0.002    0.000    0.002    0.000 arrayprint.py:399(array2stri

For sparse selections, processing time is dominated by decompression, so we can't do any better.

### sparse bool selection as zarr array

In [77]:
zix_sparse_bool = zarr.array(ix_sparse_bool)
zix_sparse_bool.info

Type,zarr.core.Array
Data type,bool
Shape,"(100000000,)"
Chunk shape,"(390625,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,100000000 (95.4M)
No. bytes stored,507131 (495.2K)
Storage ratio,197.2


In [78]:
%time zc.oindex[zix_sparse_bool]

CPU times: user 744 ms, sys: 120 ms, total: 864 ms
Wall time: 384 ms


array([     888,     9941,    15901, ..., 99988491, 99988714, 99995248])

### slice with step

In [79]:
%time np.array(c[::2])

CPU times: user 40 ms, sys: 52 ms, total: 92 ms
Wall time: 89.5 ms


array([       0,        2,        4, ..., 99999994, 99999996, 99999998])

In [80]:
%time zc.oindex[::2]

CPU times: user 1.29 s, sys: 240 ms, total: 1.53 s
Wall time: 1.24 s


array([       0,        2,        4, ..., 99999994, 99999996, 99999998])

In [81]:
%time zc.oindex[::10]

CPU times: user 588 ms, sys: 88 ms, total: 676 ms
Wall time: 411 ms


array([       0,       10,       20, ..., 99999970, 99999980, 99999990])

In [82]:
%time zc.oindex[::100]

CPU times: user 400 ms, sys: 28 ms, total: 428 ms
Wall time: 184 ms


array([       0,      100,      200, ..., 99999700, 99999800, 99999900])

In [83]:
%time zc.oindex[::1000]

CPU times: user 368 ms, sys: 52 ms, total: 420 ms
Wall time: 167 ms


array([       0,     1000,     2000, ..., 99997000, 99998000, 99999000])

In [84]:
cProfile.run('zc.oindex[::2]', sort='time')

         64607 function calls in 1.249 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.593    0.593    0.776    0.776 indexing.py:304(__init__)
        1    0.119    0.119    0.119    0.119 {built-in method numpy.core.multiarray.bincount}
     1024    0.115    0.000    0.247    0.000 core.py:822(_chunk_getitem)
     1024    0.110    0.000    0.116    0.000 core.py:997(_decode_chunk)
     1025    0.106    0.000    0.107    0.000 indexing.py:351(__iter__)
        1    0.074    0.074    0.074    0.074 {built-in method numpy.core.multiarray.arange}
        1    0.051    0.051    0.051    0.051 function_base.py:1848(diff)
        1    0.014    0.014    1.248    1.248 core.py:531(get_orthogonal_selection)
        4    0.013    0.003    0.013    0.003 {method 'reduce' of 'numpy.ufunc' objects}
     1025    0.006    0.000    0.134    0.000 indexing.py:484(__iter__)
     1024    0.004    0.000    0.008    0.000 index_

Here there are various setup operations that need to be done on the integer array, can't see way to avoid ATM.

## 2D Benchmarking

In [85]:
c.shape

(100000000,)

In [86]:
d = c.reshape(-1, 1000)
d.shape

(100000, 1000)

In [87]:
zd = zarr.array(d)
zd.info

Type,zarr.core.Array
Data type,int64
Shape,"(100000, 1000)"
Chunk shape,"(3125, 32)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,39228864 (37.4M)
Storage ratio,20.4


### bool orthogonal selection

In [88]:
ix0 = np.random.binomial(1, 0.5, size=d.shape[0]).astype(bool)
ix1 = np.random.binomial(1, 0.5, size=d.shape[1]).astype(bool)

In [89]:
%time d[np.ix_(ix0, ix1)]

CPU times: user 116 ms, sys: 16 ms, total: 132 ms
Wall time: 129 ms


array([[    1000,     1001,     1003, ...,     1994,     1995,     1999],
       [    6000,     6001,     6003, ...,     6994,     6995,     6999],
       [    8000,     8001,     8003, ...,     8994,     8995,     8999],
       ..., 
       [99991000, 99991001, 99991003, ..., 99991994, 99991995, 99991999],
       [99997000, 99997001, 99997003, ..., 99997994, 99997995, 99997999],
       [99998000, 99998001, 99998003, ..., 99998994, 99998995, 99998999]])

In [90]:
%time zd.oindex[ix0, ix1]

CPU times: user 780 ms, sys: 40 ms, total: 820 ms
Wall time: 387 ms


array([[    1000,     1001,     1003, ...,     1994,     1995,     1999],
       [    6000,     6001,     6003, ...,     6994,     6995,     6999],
       [    8000,     8001,     8003, ...,     8994,     8995,     8999],
       ..., 
       [99991000, 99991001, 99991003, ..., 99991994, 99991995, 99991999],
       [99997000, 99997001, 99997003, ..., 99997994, 99997995, 99997999],
       [99998000, 99998001, 99998003, ..., 99998994, 99998995, 99998999]])

### int orthogonal selection

In [91]:
ix0 = np.random.choice(d.shape[0], size=int(d.shape[0] * .5), replace=True)
ix1 = np.random.choice(d.shape[1], size=int(d.shape[1] * .5), replace=True)

In [92]:
%time d[np.ix_(ix0, ix1)]

CPU times: user 156 ms, sys: 28 ms, total: 184 ms
Wall time: 184 ms


array([[38408139, 38408374, 38408509, ..., 38408966, 38408223, 38408367],
       [29895139, 29895374, 29895509, ..., 29895966, 29895223, 29895367],
       [79133139, 79133374, 79133509, ..., 79133966, 79133223, 79133367],
       ..., 
       [95689139, 95689374, 95689509, ..., 95689966, 95689223, 95689367],
       [47381139, 47381374, 47381509, ..., 47381966, 47381223, 47381367],
       [20741139, 20741374, 20741509, ..., 20741966, 20741223, 20741367]])

In [93]:
%time zd.oindex[ix0, ix1]

CPU times: user 1.08 s, sys: 120 ms, total: 1.2 s
Wall time: 586 ms


array([[38408139, 38408374, 38408509, ..., 38408966, 38408223, 38408367],
       [29895139, 29895374, 29895509, ..., 29895966, 29895223, 29895367],
       [79133139, 79133374, 79133509, ..., 79133966, 79133223, 79133367],
       ..., 
       [95689139, 95689374, 95689509, ..., 95689966, 95689223, 95689367],
       [47381139, 47381374, 47381509, ..., 47381966, 47381223, 47381367],
       [20741139, 20741374, 20741509, ..., 20741966, 20741223, 20741367]])

### coordinate (point) selection

In [94]:
n = int(d.size * .1)
ix0 = np.random.choice(d.shape[0], size=n, replace=True)
ix1 = np.random.choice(d.shape[1], size=n, replace=True)
n

10000000

In [84]:
%time d[ix0, ix1]

CPU times: user 256 ms, sys: 12 ms, total: 268 ms
Wall time: 265 ms


array([ 6452573, 65841096, 70323990, ..., 44175624, 34778721, 67807976])

In [95]:
%time zd.vindex[ix0, ix1]

CPU times: user 2.43 s, sys: 148 ms, total: 2.58 s
Wall time: 2.09 s


array([55010547, 87536917, 88871707, ..., 73879431, 32878018, 25168834])

In [96]:
cProfile.run('zd.vindex[ix0, ix1]', sort='time')

         48332 function calls in 2.050 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.107    1.107    1.107    1.107 {method 'argsort' of 'numpy.ndarray' objects}
        3    0.255    0.085    0.255    0.085 indexing.py:604(<genexpr>)
        3    0.193    0.064    0.193    0.064 indexing.py:580(<genexpr>)
     1024    0.164    0.000    0.328    0.000 core.py:822(_chunk_getitem)
     1024    0.137    0.000    0.144    0.000 core.py:997(_decode_chunk)
        1    0.045    0.045    0.045    0.045 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.044    0.044    1.683    1.683 indexing.py:553(__init__)
        1    0.024    0.024    0.024    0.024 {built-in method numpy.core.multiarray.bincount}
     3072    0.021    0.000    0.021    0.000 indexing.py:645(<genexpr>)
        1    0.010    0.010    0.010    0.010 function_base.py:1848(diff)
     1025    0.010    0.000    0.034    0.000 i

Points need to be partially sorted so all points in the same chunk are grouped and processed together. This requires ``argsort`` which dominates time.

## h5py comparison

N.B., not really fair because using slower compressor, but for interest...

In [65]:
import h5py
import tempfile

In [78]:
h5f = h5py.File(tempfile.mktemp(), driver='core', backing_store=False)

In [79]:
hc = h5f.create_dataset('c', data=c, compression='gzip', compression_opts=1, chunks=zc.chunks, shuffle=True)
hc

<HDF5 dataset "c": shape (100000000,), type "<i8">

In [80]:
%time hc[:]

CPU times: user 1.16 s, sys: 172 ms, total: 1.33 s
Wall time: 1.32 s


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [81]:
%time hc[ix_sparse_bool]

CPU times: user 1.11 s, sys: 0 ns, total: 1.11 s
Wall time: 1.11 s


array([    1063,    28396,    37229, ..., 99955875, 99979354, 99995791])

In [82]:
# # this is pathological, takes minutes 
# %time hc[ix_dense_bool]

In [83]:
# this is pretty slow
%time hc[::1000]

CPU times: user 38.3 s, sys: 136 ms, total: 38.4 s
Wall time: 38.1 s


array([       0,     1000,     2000, ..., 99997000, 99998000, 99999000])